In [2]:
import librosa
import librosa.display
import os
import pickle
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import re
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.under_sampling import ClusterCentroids
from xgboost import XGBClassifier

import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint

In [3]:
with open('X_train_5.pkl', 'rb') as f:
    X_train = pickle.load(f)

In [4]:
with open('y_train_5.pkl', 'rb') as f:
    y_train = pickle.load(f)

In [5]:
with open('X_val_5.pkl', 'rb') as f:
    X_val = pickle.load(f)

In [6]:
with open('y_val_5.pkl', 'rb') as f:
    y_val = pickle.load(f)

In [7]:
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_val = (X_val - mean)/std

## GB Classifier

In [8]:
y_train = y_train.argmax(axis=1)
y_val = y_val.argmax(axis=1)

In [9]:
clf = GradientBoostingClassifier()
clf.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [10]:
print(clf.score(X_train, y_train))
print(clf.score(X_val, y_val))

0.8193333333333334
0.31241655540720964


In [11]:
y_pred = clf.predict(X_val)

In [12]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        26
           1       0.00      0.00      0.00        25
           2       0.25      0.32      0.28       179
           3       0.60      0.32      0.41       441
           4       0.14      0.49      0.22        78

    accuracy                           0.31       749
   macro avg       0.20      0.22      0.18       749
weighted avg       0.43      0.31      0.33       749



In [13]:
print(confusion_matrix(y_val, y_pred))

[[  0   0   9   7  10]
 [  0   0   7   7  11]
 [  3   2  57  60  57]
 [  8   2 139 139 153]
 [  2   1  19  18  38]]


In [14]:
lb = LabelEncoder()
with open('labels', 'rb') as f:
    lb = pickle.load(f)

In [15]:
lb.inverse_transform([0,1,2,3,4])

array(['disgust', 'fear', 'happy', 'neutral', 'sad'], dtype=object)

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
clf2 = RandomForestClassifier(random_state=42, n_estimators=100)
clf2.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [19]:
print(clf2.score(X_train, y_train))
print(clf2.score(X_val, y_val))

0.9996666666666667
0.3604806408544726


In [20]:
y_pred = clf2.predict(X_val)

In [21]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        26
           1       0.00      0.00      0.00        25
           2       0.29      0.39      0.33       179
           3       0.65      0.36      0.46       441
           4       0.17      0.56      0.26        78

    accuracy                           0.36       749
   macro avg       0.22      0.26      0.21       749
weighted avg       0.47      0.36      0.38       749

